## Use this notebook to take water bodies from the HydroPolys layer, match them to the gauge data and add them to the Named_Reservoirs shapefile

In [1]:
%matplotlib inline

import sys

sys.path.append('Scripts')
from datahandling import wofs_fuser

import datacube
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from pathlib import Path
from datacube.utils import masking
from datacube.utils import geometry 
from datacube.utils.geometry import CRS
from datacube.helpers import write_geotiff
import geopandas as gpd
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json



dc = datacube.Datacube()

/env/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
/env/lib/python3.6/site-packages/datacube/drivers/postgres/_connections.py:87: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  username=username, password=password,


## Read and fomat the polygon you want to add to the Named_Reservoirs shapefile

In [ ]:
csv = 'Library/'
gdf = gpd.read_file('HydroPolys/BLUE_ROCK.shp')
gdf

In [ ]:
#Get the centroid location of the reservoir

gdf = gdf.to_crs(epsg=4326) #4326 is the tag for lat lon format
#we need to get the lat and lon of where the dam is to call the right satellite data later
#we want the lat and lon to be columns in the data to make it easy when we query the wofs data later
#Getting the centroid from the polygon and adding a latitude and longitude column (I found this code on the internet):
def getXY(pt):
    return (pt.x, pt.y)
centroidseries = gdf['geometry'].centroid
x,y = [list(t) for t in zip(*map(getXY, centroidseries))]

gdf = gdf.assign(lon=x)
gdf = gdf.assign(lat=y)
gdf = gdf[['NAME', 'SHAPE_Leng', 'SHAPE_Area', 'lat', 'lon', 'geometry']]
gdf

In [ ]:
gauge_number_df = pd.read_csv(csv, nrows=3, escapechar='#')
column2 = list(gauge_number_df)[1]
gauge_number_df = gauge_number_df.rename(columns = {column2 : 'gauge_ID'})
ID_str = gauge_number_df.at[0, 'gauge_ID']
#ID_str = ID_str.astype(int)
#ID_str = ID_str.astype(str)
lat = gauge_number_df.at[1, 'gauge_ID']
lon = gauge_number_df.at[2, 'gauge_ID']
gdf['gauge_ID'] = ID_str
gdf['lat_gauge'] = lat
gdf['lon_gauge'] = lon
gdf['Area_calc'] = ''
gdf['staion_nam'] = column2
gdf = gdf[['NAME', 'SHAPE_Leng', 'SHAPE_Area', 'lat', 'lon', 'Area_calc', 'staion_nam', 'gauge_ID', 'lat_gauge', 'lon_gauge', 'geometry']]
gdf

In [ ]:
gdf2 = gpd.read_file('Named_Reservoirs/Named_Reservoirs9.shp')
gdf2 = gdf2.to_crs(epsg=4326)
gdf2.tail()

In [ ]:
gdf3 = gdf2.append(gdf)
gdf3.tail()

In [ ]:
gdf3.to_file('Named_Reservoirs/Named_Reservoirs9.shp')